###  Loading data for homework-04

In [106]:
import pandas as pd

github_url= 'https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/04-monitoring/data/results-gpt4o-mini.csv'

url = f'{github_url}?raw=1'
df = pd.read_csv(url)

In [107]:
df =  df.iloc[:300]

In [108]:
df.head(3)

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp


### Q1. Getting the embeddings model

In [109]:
from sentence_transformers import SentenceTransformer

model_name='multi-qa-mpnet-base-dot-v1'
embedding_model = SentenceTransformer(model_name)

/Users/zaahirdawood/Documents/llm-zoomcamp/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [110]:
answer_llm = df.iloc[0].answer_llm

results_llm = embedding_model.encode(answer_llm)

round(results_llm[0],2)

np.float32(-0.42)

### Q2. Computing the dot product

In [111]:
results_df = df.to_dict(orient='records')

results_df[0]

{'answer_llm': 'You can sign up for the course by visiting the course page at [http://mlzoomcamp.com/](http://mlzoomcamp.com/).',
 'answer_orig': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'document': '0227b872',
 'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp'}

In [112]:
def compute_similarity(record):
    answer_orig = record['answer_orig']
    answer_llm = record['answer_llm']
    
    v_llm = embedding_model.encode(answer_llm)
    v_orig = embedding_model.encode(answer_orig)
    
    return v_llm.dot(v_orig)

In [113]:
from tqdm.autonotebook import tqdm
similarity = []

for record in tqdm(results_df):
    sim = compute_similarity(record)
    similarity.append(sim)

100%|██████████| 300/300 [00:15<00:00, 19.66it/s]


In [114]:
results_df= pd.DataFrame(results_df)
results_df['cosine'] = similarity
results_df['cosine'].describe()

count    300.000000
mean      27.495996
std        6.384743
min        4.547927
25%       24.307841
50%       28.336862
75%       31.674305
max       39.476013
Name: cosine, dtype: float64

### Q3. Computing the cosine

In [115]:
norm = np.sqrt(((results_df['cosine'].values * results_df['cosine'].values).sum()))
norm_cosine = results_df['cosine'].values / norm

In [117]:
results_df['norm_cosine'] = norm_cosine

In [120]:
round(results_df['norm_cosine'].describe() * 10,2)

count    3000.00
mean        0.56
std         0.13
min         0.09
25%         0.50
50%         0.58
75%         0.65
max         0.81
Name: norm_cosine, dtype: float64

### Q4. Rouge

In [121]:
from rouge import Rouge
rouge_scorer = Rouge()